### Preprocessing

In [4]:
import numpy as np
import pandas as pd
import glob
import os
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')
#from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.decomposition import LatentDirichletAllocation as lda
from sklearn.model_selection import train_test_split

%run '/Users/Hugo/Documents/Github/HOTT_NLP_ENSAE/data.py'
%run '/Users/Hugo/Documents/Github/HOTT_NLP_ENSAE/distances.py'
%run '/Users/Hugo/Documents/Github/HOTT_NLP_ENSAE/hott.py'
%run '/Users/Hugo/Documents/Github/HOTT_NLP_ENSAE/knn_classifier.py'

#### On load le dataframe sur lequel on travaille (ici on prend un tout petit subsample)

In [5]:
data_dir = '/Users/Hugo/Documents/Cours/ENSAE/3A/S2/NLP/Projet/data/nyt-comments'
embed_path_50 = '/Users/Hugo/Documents/Cours/ENSAE/3A/S2/NLP/Projet/data/glove.6B/glove.6B.300d.txt'
df_ = pd.read_csv(glob.glob(data_dir+'/Comments*.csv')[0], encoding = 'latin1')
df_ = df_.sample(frac=1).reset_index(drop=True)
#cols = df_.columns
df_test = df_[0:1000].reset_index(drop = True)
#del df_

In [ ]:
data = load_data(df_test, embed_path_50, stemming = True, K = 70, p = 1, n_word_keep = 20)

#### Revoir la fonction qui enlève les stops words car elle ne semble pas fonctionner : pourrait expliquer les mauvaises perf

In [7]:
bow_train, bow_test, topic_train, topic_test, y_train, y_test = train_test_split(data['X'],
                                                                                 data['proportions'],
                                                                                 data['y'],
                                                                                 random_state= 0)

In [8]:
C = data['cost_T']
n_classes = len(np.unique(y_train))

In [12]:
# We could test with wmd and other distances in the file distances.py, however, the computational time may limit us
for neigh in :
    hott_test_error, hott_pred = knn(topic_train, topic_test, y_train, y_test, hott, C, n_neighbors=neigh)
    print('Pour un un nombre de voisin %s l\'erreur sur la base de test est %s'%(neigh,hott_test_error))

Pour un un nombre de voisin 7 l'erreur sur la base de test est 0.748
Pour un un nombre de voisin 9 l'erreur sur la base de test est 0.776
Pour un un nombre de voisin 11 l'erreur sur la base de test est 0.78
Pour un un nombre de voisin 13 l'erreur sur la base de test est 0.772


RecursionError: maximum recursion depth exceeded in comparison